## 1. Setup Environment

In [1]:
%pip install voyageai

In [2]:
# Import required libraries
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch
import torch.nn.functional as F
import openai
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import voyageai
import json
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

print("All libraries imported successfully!")

All libraries imported successfully!


In [3]:
# Configure API keys
from google.colab import userdata

# Get API keys from environment variables
openai.api_key = userdata.get('OPENAI_API_KEY')
voyage_api_key = userdata.get('VOYAGE_API_KEY')

# Set up Voyage API client if key exists
if voyage_api_key:
    voyageai.api_key = voyage_api_key
    print("✅ Voyage API key configured.")
else:
    print("⚠️  Voyage API key not found.")

# Configure Hugging Face token for remote LLaMA inference
hf_token = userdata.get('HF_TOKEN')
if hf_token:
    print("✅ Hugging Face token configured.")
else:
    print("⚠️  Hugging Face token not found.")

✅ Voyage API key configured.
✅ Hugging Face token configured.


## 2. Data Preparation

In [4]:
# Define model configuration
model_config = {
    'bge': {
        'enabled': True,
        'improvements': {
            'optimal_threshold': True,     # Use optimized classification threshold
            'text_preprocessing': True,    # Enhanced text preprocessing
            'text_chunking': True,        # Break text into smaller chunks
            'section_weighting': True      # Weight different resume/job sections
        }
    },
    'openai': {'enabled': False},
    'bge_m3': {'enabled': False},
    'careerbert': {'enabled': False},
    'confit': {'enabled': False},
    'voyage': {'enabled': False},
    'llama_remote': {'enabled': False}
}

print("Model configuration initialized.")

Model configuration initialized.


In [5]:
# Load the dataset
from google.colab import drive
drive.mount('/content/drive')
DATASET_PATH='/content/drive/MyDrive/AI-ML Self Learning/next_horizon/resume_job_recommendation/model-shashu2325-resume-job-matcher-lora'

# Load and sample the dataset
print("Loading dataset...")
df = pd.read_csv(f'{DATASET_PATH}/dataset.csv')

# Display dataset info
print(f"\nDataset Info:")
print(f"Total records: {len(df)}")
print(f"Columns: {list(df.columns)}")

# Create balanced sample
select_df = df[df['Decision'] == 'select']
reject_df = df[df['Decision'] == 'reject']

total_samples = 100
num_select = min(len(select_df), total_samples // 2)
num_reject = min(len(reject_df), total_samples - num_select)

# Adjust if needed
if num_select + num_reject < total_samples:
    if len(select_df) > len(reject_df):
        num_select = min(len(select_df), total_samples - num_reject)
    else:
        num_reject = min(len(reject_df), total_samples - num_select)

# Sample and combine, preserving original indices
sampled_select_df = select_df.sample(n=num_select, random_state=42)
sampled_reject_df = reject_df.sample(n=num_reject, random_state=42)
sampled_df = pd.concat([sampled_select_df, sampled_reject_df])
sampled_df = sampled_df.sample(frac=1, random_state=42)  # Shuffle but keep original index

print(f"\n✅ Final sample size: {len(sampled_df)} records")
print(f"Sample distribution: {sampled_df['Decision'].value_counts().to_dict()}")

Mounted at /content/drive
Loading dataset...

Dataset Info:
Total records: 10174
Columns: ['Role', 'Resume', 'Decision', 'Reason_for_decision', 'Job_Description']

✅ Final sample size: 100 records
Sample distribution: {'reject': 50, 'select': 50}


## 3. Model-Specific Operations

In [6]:
# Utility function for cosine similarity
def cosine_similarity(a, b):
    """Calculate cosine similarity between two vectors"""
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)
    na = np.linalg.norm(a)
    nb = np.linalg.norm(b)
    if na == 0 or nb == 0:
        return 0.0
    return float(a.dot(b) / (na * nb))

In [7]:
# BGE Model Improvements
import re
from sklearn.metrics import f1_score

def find_optimal_threshold(model_scores, true_labels, thresholds=np.arange(0.5, 0.9, 0.05)):
    """Find the optimal classification threshold"""
    best_f1 = 0
    best_threshold = 0.5

    for threshold in thresholds:
        predictions = ['select' if score > threshold else 'reject' for score in model_scores]
        f1 = f1_score(true_labels, predictions, average='weighted')

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    return best_threshold

def preprocess_text(text):
    """Enhanced text preprocessing"""
    # Remove special characters and normalize whitespace
    text = re.sub(r'[^\w\s]', ' ', text)
    text = ' '.join(text.split())

    # Convert to lowercase
    text = text.lower()

    # Remove common stopwords
    stop_words = set(['the', 'a', 'an', 'and', 'or', 'but'])
    words = [w for w in text.split() if w not in stop_words]

    return ' '.join(words)

def chunk_text(text, max_length=512):
    """Split text into smaller chunks"""
    sentences = text.split('.')
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        sentence_length = len(sentence.split())
        if current_length + sentence_length <= max_length:
            current_chunk.append(sentence)
            current_length += sentence_length
        else:
            chunks.append('. '.join(current_chunk))
            current_chunk = [sentence]
            current_length = sentence_length

    if current_chunk:
        chunks.append('. '.join(current_chunk))
    return chunks

def extract_sections(text):
    """Extract and weight different sections of the text"""
    sections = {
        'skills': re.findall(r'skills:.*?(?=\n\n|\Z)', text, re.I | re.S),
        'experience': re.findall(r'experience:.*?(?=\n\n|\Z)', text, re.I | re.S),
        'education': re.findall(r'education:.*?(?=\n\n|\Z)', text, re.I | re.S)
    }
    return sections

print("✅ BGE model improvements defined.")

✅ BGE model improvements defined.


In [8]:
# BGE Model
if model_config['bge']['enabled']:
    print("Loading BGE model...")
    try:
        model = SentenceTransformer("BAAI/bge-large-en-v1.5")

        def calculate_bge_similarity(resume_text, job_text):
            improvements = model_config['bge']['improvements']

            # Apply text preprocessing if enabled
            if improvements['text_preprocessing']:
                resume_text = preprocess_text(resume_text)
                job_text = preprocess_text(job_text)

            if improvements['text_chunking']:
                # Process text in chunks and take max similarity
                resume_chunks = chunk_text(resume_text)
                job_chunks = chunk_text(job_text)

                similarities = []
                for r_chunk in resume_chunks:
                    for j_chunk in job_chunks:
                        r_emb = model.encode(r_chunk, normalize_embeddings=True)
                        j_emb = model.encode(j_chunk, normalize_embeddings=True)
                        similarities.append(cosine_similarity(r_emb, j_emb))

                similarity = max(similarities) if similarities else 0.0

            elif improvements['section_weighting']:
                # Use weighted section matching
                weights = {'skills': 0.4, 'experience': 0.4, 'education': 0.2}
                resume_sections = extract_sections(resume_text)
                job_sections = extract_sections(job_text)

                weighted_similarity = 0
                for section, weight in weights.items():
                    if resume_sections[section] and job_sections[section]:
                        r_text = ' '.join(resume_sections[section])
                        j_text = ' '.join(job_sections[section])
                        r_emb = model.encode(r_text, normalize_embeddings=True)
                        j_emb = model.encode(j_text, normalize_embeddings=True)
                        section_sim = cosine_similarity(r_emb, j_emb)
                        weighted_similarity += section_sim * weight

                similarity = weighted_similarity

            else:
                # Basic similarity calculation
                resume_embedding = model.encode(resume_text, normalize_embeddings=True)
                job_embedding = model.encode(job_text, normalize_embeddings=True)
                similarity = cosine_similarity(resume_embedding, job_embedding)

            return similarity

        print("✅ BGE model loaded and similarity function defined.")
    except Exception as e:
        print(f"❌ Error loading BGE model: {e}")
        model_config['bge']['enabled'] = False

Loading BGE model...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

✅ BGE model loaded and similarity function defined.


In [9]:
# OpenAI Model
if model_config['openai']['enabled'] and openai.api_key:
    print("Setting up OpenAI model...")
    try:
        def get_openai_embedding(text, model="text-embedding-3-small"):
            text = text.replace("\n", " ")
            return openai.embeddings.create(input=[text], model=model).data[0].embedding

        def calculate_openai_similarity(resume_text, job_text):
            resume_embedding = get_openai_embedding(resume_text)
            job_embedding = get_openai_embedding(job_text)
            return cosine_similarity(resume_embedding, job_embedding)

        print("✅ OpenAI embedding function defined.")
    except Exception as e:
        print(f"❌ Error setting up OpenAI: {e}")
        model_config['openai']['enabled'] = False
else:
    print("⚠️ OpenAI model disabled (no API key)")
    model_config['openai']['enabled'] = False

⚠️ OpenAI model disabled (no API key)


In [10]:
# Voyage Model
if model_config['voyage']['enabled'] and voyage_api_key:
    print("Setting up Voyage model...")
    try:
        voyageai.api_key = voyage_api_key

        def calculate_voyage_similarity(resume_text, job_text, model="voyage-3-large"):
            try:
                # Create a new client instance for each request
                vo = voyageai.Client()
                # Clean and normalize texts
                resume_text = ' '.join(resume_text.split())  # Normalize whitespace
                job_text = ' '.join(job_text.split())
                # Get embeddings
                resume_embedding = vo.embed([resume_text], model=model).embeddings[0]
                job_embedding = vo.embed([job_text], model=model).embeddings[0]
                return cosine_similarity(resume_embedding, job_embedding)
            except Exception as e:
                print(f"Error in Voyage embedding calculation: {str(e)}")
                return None

        print("✅ Voyage embedding function defined.")
    except Exception as e:
        print(f"❌ Error setting up Voyage: {e}")
        model_config['voyage']['enabled'] = False
else:
    print("⚠️ Voyage model disabled (no API key)")
    model_config['voyage']['enabled'] = False

⚠️ Voyage model disabled (no API key)


In [11]:
# Remote LLaMA Model
if model_config['llama_remote']['enabled'] and hf_token:
    print("Setting up remote LLaMA inference...")
    try:
        from openai import OpenAI
        client = OpenAI(
            base_url="https://router.huggingface.co/v1",
            api_key=hf_token,
        )

        def clean_text(text):
            """Clean and prepare text for LLaMA processing"""
            # Remove any problematic characters and normalize whitespace
            text = text.replace('\n', ' ').replace('\r', ' ')
            text = ' '.join(text.split())  # Normalize whitespace
            # Truncate long texts to prevent context overflow
            max_chars = 1000
            if len(text) > max_chars:
                text = text[:max_chars] + "..."
            return text

        def calculate_llama_remote_similarity(resume_text, job_text):
            # Clean and prepare the texts
            resume_text = clean_text(resume_text)
            job_text = clean_text(job_text)

            messages = [
                {
                    "role": "system",
                    "content": """You are a CV-job matcher. Analyze the match and return ONLY a JSON with {score: <number>}.
The score should be between 0-100, indicating compatibility. Do not include any other fields or explanations."""
                },
                {
                    "role": "user",
                    "content": f"CV: {resume_text}\nJob: {job_text}"
                }
            ]

            try:
                completion = client.chat.completions.create(
                    model="meta-llama/Llama-3.1-8B-Instruct",
                    messages=messages,
                    max_tokens=32,  # Very short response needed
                    response_format={ "type": "json_object" },
                    temperature=0.1  # Lower temperature for more consistent outputs
                )
                response_text = completion.choices[0].message.content.strip()

                try:
                    # Try to parse the JSON response
                    response = json.loads(response_text)
                    score = response.get('score', 0)
                except json.JSONDecodeError:
                    # If JSON parsing fails, try to extract just the score using regex
                    import re
                    # Look for any number pattern after "score"
                    score_match = re.search(r'"score"\s*:\s*(\d+)', response_text)
                    if score_match:
                        score = int(score_match.group(1))
                    else:
                        # Try to find any number between 0-100 in the response
                        number_match = re.search(r'\b([0-9]{1,2}|100)\b', response_text)
                        if number_match:
                            score = int(number_match.group(1))
                        else:
                            print(f"Could not find score in response: {response_text}")
                            return None

                # Ensure score is within valid range
                score = max(0, min(100, int(score)))
                return score / 100.0  # Normalize to 0-1

            except Exception as e:
                print(f"Error in LLaMA remote inference: {str(e)}")
                return None

        print("✅ Remote LLaMA inference function defined.")
    except Exception as e:
        print(f"❌ Error setting up remote LLaMA: {e}")
        model_config['llama_remote']['enabled'] = False
else:
    print("⚠️ Remote LLaMA model disabled (no HF token)")
    model_config['llama_remote']['enabled'] = False

⚠️ Remote LLaMA model disabled (no HF token)


In [12]:
# BGE-M3 Model
if model_config['bge_m3']['enabled']:
    print("Loading BGE-M3 model...")
    try:
        bge_m3_model = SentenceTransformer("BAAI/bge-m3")

        def calculate_bge_m3_similarity(resume_text, job_text):
            # Clean and normalize texts
            resume_text = ' '.join(resume_text.split())
            job_text = ' '.join(job_text.split())

            # Get embeddings with BGE-M3
            resume_embedding = bge_m3_model.encode(resume_text, normalize_embeddings=True)
            job_embedding = bge_m3_model.encode(job_text, normalize_embeddings=True)
            return cosine_similarity(resume_embedding, job_embedding)

        print("✅ BGE-M3 model loaded and similarity function defined.")
    except Exception as e:
        print(f"❌ Error loading BGE-M3 model: {e}")
        model_config['bge_m3']['enabled'] = False
else:
    print("⚠️ BGE-M3 model disabled")

⚠️ BGE-M3 model disabled


In [13]:
# CareerBERT Model
if model_config['careerbert']['enabled']:
    print("Loading CareerBERT model...")
    try:
        careerbert_model = SentenceTransformer("yxchar/career-bert-base")

        def calculate_careerbert_similarity(resume_text, job_text):
            # Clean and normalize texts
            resume_text = ' '.join(resume_text.split())
            job_text = ' '.join(job_text.split())

            # Get embeddings with CareerBERT
            resume_embedding = careerbert_model.encode(resume_text, normalize_embeddings=True)
            job_embedding = careerbert_model.encode(job_text, normalize_embeddings=True)
            return cosine_similarity(resume_embedding, job_embedding)

        print("✅ CareerBERT model loaded and similarity function defined.")
    except Exception as e:
        print(f"❌ Error loading CareerBERT model: {e}")
        model_config['careerbert']['enabled'] = False
else:
    print("⚠️ CareerBERT model disabled")

⚠️ CareerBERT model disabled


In [14]:
# ConFit Model
if model_config['confit']['enabled']:
    print("Loading ConFit model...")
    try:
        confit_model = SentenceTransformer("jamescalam/resume-job-confit-base")

        def calculate_confit_similarity(resume_text, job_text):
            # Clean and normalize texts
            resume_text = ' '.join(resume_text.split())
            job_text = ' '.join(job_text.split())

            # Get embeddings with ConFit
            resume_embedding = confit_model.encode(resume_text, normalize_embeddings=True)
            job_embedding = confit_model.encode(job_text, normalize_embeddings=True)
            return cosine_similarity(resume_embedding, job_embedding)

        print("✅ ConFit model loaded and similarity function defined.")
    except Exception as e:
        print(f"❌ Error loading ConFit model: {e}")
        model_config['confit']['enabled'] = False
else:
    print("⚠️ ConFit model disabled")

⚠️ ConFit model disabled


## 4. Generate Predictions and Classifications

In [15]:
# Initialize score dictionary for each model
scores = {
    'bge': [],
    'openai': [],
    'bge_m3': [],
    'careerbert': [],
    'confit': [],
    'voyage': [],
    'llama_remote': []
}

# Process each resume-job pair
print("🚀 Calculating similarity scores for all enabled models...")
for index, row in sampled_df.iterrows():
    resume_text = str(row['Resume']) if pd.notna(row['Resume']) else ""
    job_text = str(row['Job_Description']) if pd.notna(row['Job_Description']) else ""

    print(f"\n📋 Processing pair {index+1}/{len(sampled_df)}")

    # Calculate scores for each enabled model
    if model_config['bge']['enabled']:
        try:
            scores['bge'].append(calculate_bge_similarity(resume_text, job_text))
        except Exception as e:
            scores['bge'].append(None)
            print(f"❌ BGE error: {e}")
    else:
        scores['bge'].append(None)

    if model_config['openai']['enabled']:
        try:
            scores['openai'].append(calculate_openai_similarity(resume_text, job_text))
        except Exception as e:
            scores['openai'].append(None)
            print(f"❌ OpenAI error: {e}")
    else:
        scores['openai'].append(None)

    if model_config['bge_m3']['enabled']:
        try:
            scores['bge_m3'].append(calculate_bge_m3_similarity(resume_text, job_text))
        except Exception as e:
            scores['bge_m3'].append(None)
            print(f"❌ BGE-M3 error: {e}")
    else:
        scores['bge_m3'].append(None)

    if model_config['careerbert']['enabled']:
        try:
            scores['careerbert'].append(calculate_careerbert_similarity(resume_text, job_text))
        except Exception as e:
            scores['careerbert'].append(None)
            print(f"❌ CareerBERT error: {e}")
    else:
        scores['careerbert'].append(None)

    if model_config['confit']['enabled']:
        try:
            scores['confit'].append(calculate_confit_similarity(resume_text, job_text))
        except Exception as e:
            scores['confit'].append(None)
            print(f"❌ ConFit error: {e}")
    else:
        scores['confit'].append(None)

    if model_config['voyage']['enabled']:
        try:
            scores['voyage'].append(calculate_voyage_similarity(resume_text, job_text))
        except Exception as e:
            scores['voyage'].append(None)
            print(f"❌ Voyage error: {e}")
    else:
        scores['voyage'].append(None)

    if model_config['llama_remote']['enabled']:
        try:
            scores['llama_remote'].append(calculate_llama_remote_similarity(resume_text, job_text))
        except Exception as e:
            scores['llama_remote'].append(None)
            print(f"❌ LLaMA Remote error: {e}")
    else:
        scores['llama_remote'].append(None)

print("\n✅ Score calculation complete!")

🚀 Calculating similarity scores for all enabled models...

📋 Processing pair 7548/100

📋 Processing pair 2644/100

📋 Processing pair 4814/100

📋 Processing pair 2877/100

📋 Processing pair 1746/100

📋 Processing pair 1176/100

📋 Processing pair 6368/100

📋 Processing pair 4668/100

📋 Processing pair 3673/100

📋 Processing pair 6143/100

📋 Processing pair 7891/100

📋 Processing pair 5265/100

📋 Processing pair 1774/100

📋 Processing pair 4468/100

📋 Processing pair 1524/100

📋 Processing pair 4507/100

📋 Processing pair 1458/100

📋 Processing pair 2815/100

📋 Processing pair 6717/100

📋 Processing pair 2858/100

📋 Processing pair 5058/100

📋 Processing pair 8698/100

📋 Processing pair 9656/100

📋 Processing pair 2812/100

📋 Processing pair 5435/100

📋 Processing pair 4140/100

📋 Processing pair 2751/100

📋 Processing pair 6016/100

📋 Processing pair 188/100

📋 Processing pair 3052/100

📋 Processing pair 5246/100

📋 Processing pair 7154/100

📋 Processing pair 8559/100

📋 Processing pair 

In [18]:
# Add scores to DataFrame and generate predictions
threshold = 0.5
print(f"Using classification threshold: {threshold}")

# Function to make predictions
def make_prediction(score, threshold=0.5):
    if score is None or pd.isna(score):
        return 'unknown'
    return 'select' if score > threshold else 'reject'

# Add scores and predictions to DataFrame
for model in scores.keys():
    if model_config[model]['enabled']:
        sampled_df[f'{model}_similarity'] = scores[model]
        sampled_df[f'{model}_prediction'] = [make_prediction(s, threshold) for s in scores[model]]

# Create display columns, moving Reason_for_decision to the end
display_columns = ['Role', 'Decision']
for model in scores.keys():
    if model_config[model]['enabled']:
        display_columns.extend([f'{model}_similarity', f'{model}_prediction'])
#display_columns.append('Reason_for_decision')  # Add Reason_for_decision at the end

# Create comparative table with original index
comparison_table = sampled_df[display_columns]

print("\n📊 Comparative Table of Similarity Scores and Predictions (with original dataset indices):")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
print(comparison_table.to_string())

Using classification threshold: 0.5

📊 Comparative Table of Similarity Scores and Predictions (with original dataset indices):
                             Role Decision  bge_similarity bge_prediction
7547            Robotics Engineer   reject          0.5991         select
2643        E-commerce Specialist   reject          0.6233         select
4813               Cloud Engineer   reject          0.6788         select
2876        E-commerce Specialist   select          0.6098         select
1745             Network Engineer   select          0.7698         select
1175              Product Manager   select          0.6758         select
6367       Database Administrator   select          0.5901         select
4667        IT Support Specialist   reject          0.4496         reject
3672         Mobile App Developer   select          0.5776         select
6142                  QA Engineer   select          0.5740         select
7890              DevOps Engineer   select          0.6730 

In [17]:
# Calculate performance metrics for each enabled model
print("\n📈 Model Performance Metrics:")
print("=" * 80)

for model in scores.keys():
    if model_config[model]['enabled']:
        print(f"\n🔍 {model.upper()} Model Performance:")
        print("-" * 50)

        # Filter out unknown predictions
        valid_mask = sampled_df[f'{model}_prediction'] != 'unknown'
        valid_df = sampled_df[valid_mask]

        if len(valid_df) > 0:
            accuracy = accuracy_score(valid_df['Decision'], valid_df[f'{model}_prediction'])
            precision, recall, f1, _ = precision_recall_fscore_support(
                valid_df['Decision'],
                valid_df[f'{model}_prediction'],
                average='weighted'
            )

            print(f"Accuracy: {accuracy:.4f}")
            print(f"Precision: {precision:.4f}")
            print(f"Recall: {recall:.4f}")
            print(f"F1-Score: {f1:.4f}")
            print(f"Valid predictions: {len(valid_df)}/{len(sampled_df)}")

            print("\nDetailed Classification Report:")
            print(classification_report(valid_df['Decision'], valid_df[f'{model}_prediction']))
        else:
            print("No valid predictions available")


📈 Model Performance Metrics:

🔍 BGE Model Performance:
--------------------------------------------------
Accuracy: 0.4800
Precision: 0.4113
Recall: 0.4800
F1-Score: 0.3552
Valid predictions: 100/100

Detailed Classification Report:
              precision    recall  f1-score   support

      reject       0.33      0.04      0.07        50
      select       0.49      0.92      0.64        50

    accuracy                           0.48       100
   macro avg       0.41      0.48      0.36       100
weighted avg       0.41      0.48      0.36       100

